# Tobig's 정규세션 10주차 음성 과제 1

- 다음 5개의 질문에 **단답형**으로 답해주세요.

## Q1. 시간에 따른 소리의 진폭을 나타낸 그래프를 영어로 무엇이라고 하나요?

A1. Waveform

## Q2. 샘플링 주파수는 원래 신호의 최고 주파수의 2배 이상이 되어야 원래 신호로 복구할 수 있다는 정리의 이름을 영어로 무엇이라고 하나요?

A2. Nyquist-Shannon Sampling Theorem

## Q3. 소리를 시각화한 것으로, 단시간 푸리에 변환을 통해 시간, 주파수, 진폭 정보를 모두 담고 있는 것을 영어로 무엇이라고 하나요?

A3. Spectrogram

## Q4. 인간의 청각 시스템에 맞게 Mel-scale을 적용하여 주파수를 조절한 스펙트로그램을 영어로 무엇이라고 하나요?

A4. Mel-spectrogram

## Q5. Speech 정규세션 강의자가 수업 시작할 때 5명을 랜덤으로 지목하여 질문하겠다고 했으나, 실제로는 그보다 적은 n명에게 질문하였습니다. n의 값은 무엇입니까?

A5. 0

# Tobig's 정규세션 10주차 음성 과제 2

1. train 폴더 내의 2,000개 음원을 이용하여 음성 분류 모델을 만들어보세요.
2. 음성 분류 모델을 이용하여 test 폴더 내의 300개 음원을 분류하세요.

In [54]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [55]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.regularizers import l2

In [56]:
data_path = '/content/drive/MyDrive/Dataset/train'
files = os.listdir(data_path)
max_len = 500

X = []
y = []

for label in range(10):
    class_files = [file for file in files if int(file[-5]) == label]
    class_files = class_files[:200]
    for file in class_files:
        file_path = os.path.join(data_path, file)
        audio, sr = librosa.load(file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)  # MFCC 추출

        pad_width = max_len - mfccs.shape[1]
        mfccs_padded = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

        X.append(mfccs_padded)
        y.append(label)

X = np.array(X)
y = np.array(y)


In [57]:
X = np.expand_dims(X, axis=-1)
y = np.expand_dims(y, axis=-1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])


In [62]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=40, batch_size=32)

loss, accuracy = model.evaluate(X_val, y_val)
print("Validation Accuracy:", accuracy)


Epoch 1/40
50/50 [==============================] - 2s 10ms/step - loss: 0.0562 - accuracy: 0.9869 - val_loss: 4.2070 - val_accuracy: 0.7275
Epoch 2/40
50/50 [==============================] - 0s 6ms/step - loss: 0.0468 - accuracy: 0.9887 - val_loss: 4.2708 - val_accuracy: 0.7050
Epoch 3/40
50/50 [==============================] - 0s 6ms/step - loss: 0.0456 - accuracy: 0.9912 - val_loss: 4.5094 - val_accuracy: 0.7300
Epoch 4/40
50/50 [==============================] - 0s 6ms/step - loss: 0.0303 - accuracy: 0.9869 - val_loss: 4.6186 - val_accuracy: 0.7400
Epoch 5/40
50/50 [==============================] - 0s 6ms/step - loss: 0.0266 - accuracy: 0.9937 - val_loss: 4.4563 - val_accuracy: 0.7425
Epoch 6/40
50/50 [==============================] - 0s 6ms/step - loss: 0.0216 - accuracy: 0.9925 - val_loss: 4.4558 - val_accuracy: 0.7475
Epoch 7/40
50/50 [==============================] - 0s 6ms/step - loss: 0.0444 - accuracy: 0.9900 - val_loss: 4.3837 - val_accuracy: 0.7375
Epoch 8/40
50/50 [=

In [63]:
test_data_path = '/content/drive/MyDrive/Dataset/test'

def load_test_data(data_path, max_len=500):
    files = os.listdir(data_path)
    X_test = []

    for file in files:
        file_path = os.path.join(data_path, file)
        audio, sr = librosa.load(file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)  # MFCC 추출
        pad_width = max_len - mfccs.shape[1]
        mfccs_padded = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        X_test.append(mfccs_padded)

    X_test = np.array(X_test)
    X_test = np.expand_dims(X_test, axis=-1)
    return X_test

X_test = load_test_data(test_data_path)

predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

print("Test result:")
print(' '.join(map(str, predicted_labels)))


10/10 [==============================] - 0s 2ms/step
Test result:
7 7 7 6 8 1 8 5 8 4 5 9 0 4 5 3 3 6 3 6 0 5 7 1 4 7 1 3 5 0 6 6 5 4 9 7 7 1 7 1 4 7 0 5 3 6 5 9 5 5 5 1 7 4 5 6 6 5 6 7 6 2 8 1 1 4 0 7 3 3 3 0 3 5 3 5 4 7 5 9 4 5 7 1 4 8 2 1 7 8 1 9 5 6 3 6 3 1 9 4 2 7 4 6 6 3 6 7 3 3 1 6 5 3 3 5 8 7 6 6 3 3 7 3 6 5 5 3 6 0 3 5 0 3 3 0 8 5 3 3 0 3 3 4 2 9 5 2 6 0 5 5 5 3 3 6 1 3 1 7 3 3 3 5 8 5 9 6 5 4 0 4 9 3 7 6 4 3 9 3 6 5 5 3 6 5 6 5 6 1 9 7 7 5 1 0 1 9 0 1 6 5 6 3 8 3 4 2 8 1 9 3 0 5 0 3 1 5 4 0 6 6 6 3 3 6 3 3 3 3 4 1 6 7 5 0 3 3 4 4 7 6 1 4 6 0 6 6 8 0 8 7 2 1 3 5 3 3 6 0 4 4 3 4 4 3 4 0 8 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3
